# Nombre
*   Marco Antonio Bustamante Calderón

# Tarea 1 Deep Learning: Redes Convolucionales
En esta tarea vas a experimentar con redes convolucionales y su comparación con redes MLP. Lo primero que haremos es importar los paquetes importantes para nuestro notebook e inicializar el gpu. Asegúrate que tu notebook se ejecuta sobre un GPU. Si es así, al ejecutar la siguiente celda debería salirte el mensaje "cuda".

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
import numpy as np
import random
import matplotlib.pyplot as plt


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


A continuación hay un conjunto de funciones que puedes usar para hacer tus experimentos. Los detalles de cada función están como comentario antes del código de la función.

In [2]:
#Esta función permite inicializar todas las semillas de números pseudoaleatorios.
# Puedes usar esta función para resetear los generadores de números aleatorios
def iniciar_semillas():
  SEED = 1234

  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True

#Función para computar el accuracy. Se asume que predicciones y etiquetas son tensores en el GPU
def calculate_accuracy(y_pred, y):
  top_pred = y_pred.argmax(1, keepdim=True)
  correct = top_pred.eq(y.view_as(top_pred)).sum()
  acc = correct.float()/y.shape[0]
  return acc

#Función para entrenar una época de un modelo. Recibe como parámetros
#     -model: una red neuronal
#     -iterator: un iterador de la data a usar para el entrenamiento (generalmente creado con un DataLoader)
#     -optimizer: el optimizador para el entrenamiento
#     -criterion: la función de loss
#     -device: dispositivo a usar para el entrenamiento
#
#Devuelve el loss promedio y el accuracy promedio de la época (promedio de todos los batches)
def train_one_epoch(model, iterator, optimizer, criterion, device):
  epoch_loss = 0
  epoch_acc = 0

  #We have to set the neural network in training mode. This is because during
  #training, we need gradients and complementary data to ease the computation
  model.train()

  #Training loop
  for (x, y) in iterator:
    x = x.to(device) #Data
    y = y.long().to(device) #Labels

    optimizer.zero_grad() #Clean gradients

    y_pred = model(x) #Feed the network with data

    loss = criterion(y_pred, y) #Compute the loss

    acc = calculate_accuracy(y_pred, y) #Compute the accuracy

    loss.backward() #Compute gradients

    optimizer.step() #Apply update rules

    epoch_loss += loss.item()
    epoch_acc += acc.item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator)

#Función que evalúa una red neuronal con un conjunto de datos de prueba. Recibe como parámetros
#     -model: una red neuronal
#     -iterator: un iterador de la data a usar para el entrenamiento (generalmente creado con un DataLoader)
#     -criterion: la función de loss
#     -device: dispositivo a usar para el entrenamiento
#Devuelve el loss promedio y el accuracy promedio de la época (promedio de todos los batches)
def evaluate(model, iterator, criterion, device):
  epoch_loss = 0
  epoch_acc = 0

  #We put the network in testing mode
  #In this mode, Pytorch doesn't use features only reserved for
  #training (dropout for instance)
  model.eval()

  with torch.no_grad(): #disable the autograd engine (save computation and memory)

    for (x, y) in iterator:
      x = x.to(device)
      y = y.long().to(device)

      y_pred= model(x)

      loss = criterion(y_pred, y)

      acc = calculate_accuracy(y_pred, y)

      epoch_loss += loss.item()
      epoch_acc += acc.item()
  return epoch_loss / len(iterator), epoch_acc / len(iterator)

#Calcula el tiempo transcurrido entre dos timestamps
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

#Esta función realiza el entrenamiento completo de una red. Recibe como parámetros:
#     -network: la red neuronal
#     -optimizer: el optimizador para entrenamiento
#     -train_loader: el dataloader de datos de entrenamiento
#     -tes_loader: el dataloader de datos de prueba
#     -name: nombre a usar para guardar en disco la red con el mejor accuracy

def train_complete(network, optimizer, train_loader, test_loader, name):

  #Se envían la red y la función de loss al GPU
  network = network.to(device)
  criterion = nn.CrossEntropyLoss()
  criterion = criterion.to(device)

  #Fijar el entrenamiento en 10 épocas siempre
  EPOCHS = 10

  best_valid_acc = float('-inf')
  n_params = sum(p.numel() for p in network.parameters() if p.requires_grad)
  print(f'The model has {n_params:,} trainable parameters')

  for epoch in range(EPOCHS):

    start_time = time.time()

    #Train + validation cycles
    train_loss, train_acc = train_one_epoch(network, train_loader, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(network, test_loader, criterion, device)

    #Si encontramos un modelo con accuracy de validación mayor, lo guardamos
    if valid_acc > best_valid_acc:
     best_valid_acc = valid_acc
     torch.save(network.state_dict(), f'{name}.pt')

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  #Cuando se termina el entrenamiento, cargamos el mejor modelo guardado y calculamos el accuracy de prueba
  network.load_state_dict(torch.load(f'{name}.pt'))

  test_loss , test_acc = evaluate(network, test_loader, criterion, device)
  print(f'Test Loss: {test_loss:.3f} | Mejor test acc: {test_acc*100:.2f}%')

Para nuestros experimentos usaremos el dataset CIFAR10, que consta de imágenes RGB de 32x32 píxeles que representan distintos objetos y animales clasificados en 10 clases. Inicializamos los hiperparámetros y cargamos los conjuntos de datos de entrenamiento y prueba. También se crean los dataloaders correspondientes.

In [3]:
examples = enumerate(test_loader) #Crea un iterador sobre la lista de batches

batchId, (exampleData, exampleTargets) = next(examples) #Obtenemos el siguiente batch
print('Numero de batch: {}'.format(batchId))
print('Batch Shape', exampleData.shape)
print('Target Shape', exampleTargets.shape)

#Mostramos los primeros 16 elementos del batch con su etiqueta
plt.figure()
for i in range(16):
  plt.subplot(4, 4, i+1)
  plt.tight_layout()
  img = exampleData[i] / 2 + 0.5 # unnormalize
  img = img.permute(1, 2, 0)
  plt.imshow(img, interpolation='none')
  plt.title(class_names[exampleTargets[i]])
  plt.xticks([])
  plt.yticks([])

NameError: name 'test_loader' is not defined

In [ ]:
# Hyper-parameters
#mab: Se modifican hiperparámetros
input_size = 3*32*32
#input_size = 784
num_classes = 10
batch_size_train = 1000
batch_size_test = 1000
momentum = 0.9
log_interval = 100
learning_rate=0.01
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='data', train=True,
                                        download=True, transform=transform)


test_dataset = torchvision.datasets.CIFAR10(root='data', train=False,
                                       download=True, transform=transform)

# Data loader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size_train,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size_test,
                                         shuffle=False)

class_names = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Parte 1

Crea una arquitectura de red neuronal MLP que procese datos con la forma Bx3x32x32, donde B representa el tamaño del batch, y que cuente con 10 neuronas en la capa de salida. Para ello, primero transforma el tensor de entrada de la forma Bx3x32x32 a Bx(33232) para que pueda ser utilizado en una capa lineal. Además, utiliza la función ReLU como activación en todas las capas ocultas y la función softmax en la capa de salida. Ten en cuenta que si aplicas nn.CrossEntropyLoss, no es necesario añadir ninguna función adicional en la última capa de la red.


In [ ]:
# @title Texto de título predeterminado
#Creamos la red neuronal
class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    # Define aquí las capas de tu red

    self.fc1 = nn.Linear(3*32*32, 250) # 784 x 250 + 250 (bias: extra parameters)
    self.fc2 = nn.Linear(250, 100) # 250 x 100 + 100
    self.fc3 = nn.Linear(100, output_dim) # 100 x 10 + 10..

  def forward(self, x):
    #Implementa la función forward usando la función de activación ReLU en las capas ocultas
    batch_size = x.shape[0]
    #Flatten the input
    input = x.view(batch_size, -1)

    #Use ReLU as activation function
    h_1 = F.relu(self.fc1(input))
    h_2 = F.relu(self.fc2(h_1))
    y_pred = self.fc3(h_2)

    #Our network returns the output of the final layer but also the output of the hidden layer
    return y_pred



    import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # Transformación para capa lineal
        self.flatten = nn.Flatten()
        # Capas ocultas
        self.fc1 = nn.Linear(3 * 32 * 32, 250)
        self.fc2 = nn.Linear(250, 100)
        self.fc3 = nn.Linear(100,num_classes)
        # Función de activación: ReLU
        self.relu = nn.ReLU()

    def forward(self, input):
        input_flat = self.flatten(input)
        # Uso de ReLU
        h_1 = self.relu(self.fc1(input_flat))
        h_2 = self.relu(self.fc2(h_1))
        # Capa de salida (no aplicar softmax si se usa nn.CrossEntropyLoss)
        y_pred = self.fc3(h_2)
        return y_pred

# Ejemplo de uso:
model = MLP()


Para ejecutar el entrenamiento de tu primer modelo, ejecuta la siguiente celda y debería reportarte el accuracy de prueba de este primer experimento. Cuánto obtienes de accuracy de test?

In [ ]:
iniciar_semillas() # Se inicializan las semillas
network = MLP() # Creas la red
optimizer = optim.Adam(network.parameters(), lr=learning_rate) #Creas el optimizador
train_complete(network, optimizer, train_loader, test_loader, 'mlp') #Entrenas la red. IMPORTANTE: el número de épocas siempre debe ser el mismo (internamente son 10 épocas)

The model has 794,360 trainable parameters
Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 2.186 | Train Acc: 25.42%
	 Val. Loss: 1.806 |  Val. Acc: 35.22%
Epoch: 02 | Epoch Time: 0m 12s
	Train Loss: 1.742 | Train Acc: 37.35%
	 Val. Loss: 1.663 |  Val. Acc: 39.62%
Epoch: 03 | Epoch Time: 0m 12s
	Train Loss: 1.602 | Train Acc: 42.65%
	 Val. Loss: 1.577 |  Val. Acc: 43.50%
Epoch: 04 | Epoch Time: 0m 12s
	Train Loss: 1.510 | Train Acc: 46.27%
	 Val. Loss: 1.514 |  Val. Acc: 46.60%
Epoch: 05 | Epoch Time: 0m 12s
	Train Loss: 1.432 | Train Acc: 49.21%
	 Val. Loss: 1.492 |  Val. Acc: 47.43%
Epoch: 06 | Epoch Time: 0m 12s
	Train Loss: 1.374 | Train Acc: 51.54%
	 Val. Loss: 1.459 |  Val. Acc: 48.75%
Epoch: 07 | Epoch Time: 0m 12s
	Train Loss: 1.318 | Train Acc: 53.41%
	 Val. Loss: 1.439 |  Val. Acc: 49.00%
Epoch: 08 | Epoch Time: 0m 12s
	Train Loss: 1.271 | Train Acc: 55.03%
	 Val. Loss: 1.453 |  Val. Acc: 49.57%
Epoch: 09 | Epoch Time: 0m 12s
	Train Loss: 1.236 | Train Acc: 56.52%
	 Val. Loss: 1.

# Parte 2
En esta parte vas a comparar el accuracy de test de la primera red que creaste (red multilayer perceptron) y los accuracys obtenidos usando redes convolucionales. Para completar esta parte debes hacer lo siguiente:



*   Implementar la red convolucional: Crea una arquitectura que reciba imágenes RGB de 32x32 y tenga 10 neuronas de salida. Usa ReLU como función de activación , softmax en la capa de salida, y añade capas de max pooling para reducir el tamaño de los mapas de características.
*   Entrena cada red usando la función "train_complete" usando el mismo optimizador siempre.
*   Registra los accuracys que obtengas. Prueba distintas combinaciones de hiperparametros y arquitecturas para obtener los mejores resultados posibles. Evita utilizar redes muy profundas ya que estas pueden tener mayor dificultad para aprender.
*   Discute los resultados obtenidos. Incluye en tu analisis la respuesta a las siguientes preguntas:
    * Teóricamente, ¿qué tipo de red debería obtener mejores resultados al tratar con imágenes: una red convolucional o una MLP? ¿Por qué?
    * Según tus resultados, ¿qué arquitectura funcionó mejor? ¿Por qué crees que esto ocurrió así?
    * Antes mencionamos que las redes muy profundas son más difíciles de entrenar. ¿Al probar distintas arquitecturas, encontraste que se daba este fenómeno? ¿Por qué crees que ocurre esto? ¿Cómo podrías intentar solucionarlo?



In [ ]:
#Creamos la red neuronal
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    # Define aquí las capas de tu red
    # Capas convolucionales
    ## Toma imágenes de 3 canales RGB y produce 16 mapas > 32 mapas > 64 mapas
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    self.conv3 = nn.Conv2d(32, 64, kernel_size=3)

    # Capas densas (fully-connected)
    self.fc1 = nn.Linear(64*6*6, 256)
    self.fc2 = nn.Linear(256,10)


  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2)) # de 3 × 32 × 32 → 16 × 16 × 16
    x = F.relu(F.max_pool2d(self.conv2(x), 2)) # de 16 × 16 × 16 → 32 × 8 × 8
    x = F.relu(self.conv3(x)) # de 32 × 8 × 8 → 64 × 6 × 6
    x = x.view(-1, 64*6*6) # Aplana el vector
    x = F.relu(self.fc1(x)) # de 1600 a 256
    x = self.fc2(x) # de 256 a 10
    return F.log_softmax(x) # función de densidad de probabilidades logaritmicas

In [ ]:
iniciar_semillas() # Se inicializan las semillas
network = CNN() # Creas la red
optimizer = optim.Adam(network.parameters(), lr=learning_rate) #Creas el optimizador
train_complete(network, optimizer, train_loader, test_loader, 'cnn') #Entrenas la red. IMPORTANTE: el número de épocas siempre debe ser el mismo (internamente son 10 épocas)

The model has 616,234 trainable parameters


/tmp/ipython-input-1268658930.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x) # función de densidad de probabilidades logaritmicas


Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 1.817 | Train Acc: 32.50%
	 Val. Loss: 1.465 |  Val. Acc: 46.38%
Epoch: 02 | Epoch Time: 0m 15s
	Train Loss: 1.342 | Train Acc: 51.49%
	 Val. Loss: 1.242 |  Val. Acc: 55.15%
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 1.178 | Train Acc: 57.78%
	 Val. Loss: 1.116 |  Val. Acc: 59.90%
Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: 1.029 | Train Acc: 63.48%
	 Val. Loss: 1.033 |  Val. Acc: 63.05%
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.944 | Train Acc: 66.42%
	 Val. Loss: 1.005 |  Val. Acc: 64.40%
Epoch: 06 | Epoch Time: 0m 14s
	Train Loss: 0.844 | Train Acc: 70.20%
	 Val. Loss: 1.005 |  Val. Acc: 64.41%
Epoch: 07 | Epoch Time: 0m 15s
	Train Loss: 0.796 | Train Acc: 72.00%
	 Val. Loss: 0.965 |  Val. Acc: 67.17%
Epoch: 08 | Epoch Time: 0m 14s
	Train Loss: 0.704 | Train Acc: 75.05%
	 Val. Loss: 0.972 |  Val. Acc: 66.44%
Epoch: 09 | Epoch Time: 0m 14s
	Train Loss: 0.645 | Train Acc: 77.16%
	 Val. Loss: 0.987 |  Val. Acc: 66.88%
Epoch: 10 | Epoch T

# Parte 3
En esta parte, probaremos qué ocurre si se eliminan las capas de max pooling de la red que creaste en la parte 2. Para completar esta sección, debes hacer lo siguiente:

*   Copia la implementación de la red convolucional y elimina las capas de pooling. Intenta cambiar lo menos posible la implementación, solo adapta las dimensiones de la red para ajustarlas tras haber removido el pooling.
*   Entrena cada red usando la función "train_complete" usando el mismo optimizador siempre.
*   Registra los accuracys que obtengas.
*   Discute los resultados obtenidos. Incluye en tu análisis la respuesta a las siguientes preguntas:
    * ¿Cómo se comparan los resultados obtenidos con los del modelo de la parte anterior? ¿Cómo explicas lo obtenido experimentalmente? ¿Cómo varía la cantidad de parámetros del modelo de la parte anterior en comparación con este?
    * ¿Cuál es la utilidad de las capas de pooling? Si utilizáramos imágenes de mayor tamaño, ¿crees que el pooling sería más o menos necesario? Justifica tu respuesta.

In [ ]:
#Creamos la red neuronal
class CNN_sin_pooling(nn.Module):
  def __init__(self):
    super(CNN_sin_pooling, self).__init__()
    ## Toma imágenes de 3 canales RGB y produce 16 mapas > 32 mapas > 64 mapas
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    self.conv3 = nn.Conv2d(32, 64, kernel_size=3)

    # Capas densas (fully-connected)
    self.fc1 = nn.Linear(64*30*30, 256)
    self.fc2 = nn.Linear(256,10)


  def forward(self, x):
    x = F.relu(self.conv1(x)) # Removed the extra '2'
    x = F.relu(self.conv2(x)) # de 16 × 32 × 32 → 32 × 32 × 32
    x = F.relu(self.conv3(x)) # de 32 × 32 × 32 → 64 × 30 × 30
    x = x.view(-1, 64*30*30) # Aplana el vector
    x = F.relu(self.fc1(x)) # de 1600 a 256
    x = self.fc2(x) # de 256 a 10
    return F.log_softmax(x) # función de densidad de probabilidades logaritmicas# Define aquí las capas de tu red

In [ ]:
iniciar_semillas() # Se inicializan las semillas
network = CNN_sin_pooling() # Creas la red
optimizer = optim.Adam(network.parameters(), lr=learning_rate) #Creas el optimizador
train_complete(network, optimizer, train_loader, test_loader, 'CNN_sin_pooling') #Entrenas la red. IMPORTANTE: el número de épocas siempre debe ser el mismo (internamente son 10 épocas)

The model has 14,772,010 trainable parameters


/tmp/ipython-input-3169413019.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x) # función de densidad de probabilidades logaritmicas# Define aquí las capas de tu red


Epoch: 01 | Epoch Time: 0m 20s
	Train Loss: 3.060 | Train Acc: 9.92%
	 Val. Loss: 2.303 |  Val. Acc: 10.00%
Epoch: 02 | Epoch Time: 0m 20s
	Train Loss: 2.303 | Train Acc: 10.04%
	 Val. Loss: 2.303 |  Val. Acc: 10.00%
Epoch: 03 | Epoch Time: 0m 21s
	Train Loss: 2.303 | Train Acc: 9.89%
	 Val. Loss: 2.303 |  Val. Acc: 10.00%
Epoch: 04 | Epoch Time: 0m 20s
	Train Loss: 2.303 | Train Acc: 9.74%
	 Val. Loss: 2.303 |  Val. Acc: 10.00%
Epoch: 05 | Epoch Time: 0m 21s
	Train Loss: 2.303 | Train Acc: 9.72%
	 Val. Loss: 2.303 |  Val. Acc: 10.00%
Epoch: 06 | Epoch Time: 0m 21s
	Train Loss: 2.303 | Train Acc: 10.02%
	 Val. Loss: 2.303 |  Val. Acc: 10.00%
Epoch: 07 | Epoch Time: 0m 20s
	Train Loss: 2.303 | Train Acc: 9.85%
	 Val. Loss: 2.303 |  Val. Acc: 10.00%
Epoch: 08 | Epoch Time: 0m 20s
	Train Loss: 2.303 | Train Acc: 10.04%
	 Val. Loss: 2.303 |  Val. Acc: 10.00%
Epoch: 09 | Epoch Time: 0m 20s
	Train Loss: 2.303 | Train Acc: 9.70%
	 Val. Loss: 2.303 |  Val. Acc: 10.00%
Epoch: 10 | Epoch Time: 0

# Parte 4

En esta parte probaremos la arquitectura de ResNet y como este resuelve el problema del desvanecimiento de gradiente. Para eso ejecuta las siguientes celdas y responde las siguientes preguntas:

* ¿Cómo se comparan los resultados con los de las partes anteriores? ¿Cómo varía la cantidad de parámetros?
* ¿Cómo se estructuran las conexiones en ResNet para abordar el problema del desvanecimiento del gradiente y qué papel juegan las "skip connections" en esta arquitectura?


In [ ]:
from torchvision.models import resnet34

In [ ]:
m = resnet34()
m.fc = nn.Linear(512, 10)

In [ ]:
iniciar_semillas() # Se inicializan las semillas
network = m # Creas la red
optimizer = optim.Adam(network.parameters(), lr=learning_rate) #Creas el optimizador
train_complete(network, optimizer, train_loader, test_loader, 'resnet') #Entrenas la red. IMPORTANTE: el número de épocas siempre debe ser el mismo (internamente son 10 épocas)